In [15]:
import json

#Loading Data
with open("durgapur.json") as file:#intents.json
	data = json.load(file)

In [16]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [17]:
docs_x = []
docs_y = []
labels= []

In [18]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string
#from nltk.stem.lancaster import LancasterStemmer

#stemming good?
#stemmer = LancasterStemmer()
rt = RegexpTokenizer(r'[^\W_]+|[^\W_\s]+')
stop = set(stopwords.words('english') + list(string.punctuation))
#Looping through our data
for intent in data['intents']:
  words = []
  for pattern in intent['patterns']:
    pattern = pattern.lower()
    #print(pattern)
    #Creating a list of words
    ###wrds = rt.tokenize(pattern)
    wrds = [i for i in rt.tokenize(pattern) if i not in stop]
    words.append(wrds)
  docs_x.append(words)
  docs_y.append(intent['tag'])

In [19]:
print(docs_x)
print(docs_y)
print(len(docs_y))

[[['skin', 'rash'], ['dischromic', 'patches'], ['itching'], ['nodal', 'skin', 'eruptions']], [['chills'], ['shivering'], ['continuous', 'sneezing']], [['vomiting'], ['chest', 'pain'], ['acidity'], ['ulcers', 'tongue'], ['stomach', 'pain'], ['cough']], [['vomiting'], ['nausea'], ['yellowish', 'skin'], ['abdominal', 'pain'], ['loss', 'appetite'], ['yellowing', 'eyes'], ['itching']], [['burning', 'micturition'], ['skin', 'rash'], ['spotting', 'urination'], ['stomach', 'pain'], ['itching']], [['vomiting'], ['indigestion'], ['passage', 'gases'], ['loss', 'appetite'], ['abdominal', 'pain'], ['internal', 'itching']], [['extra', 'marital', 'contacts'], ['patches', 'throat'], ['high', 'fever'], ['muscle', 'wasting']], [['lethargy'], ['increased', 'appetite'], ['fatigue'], ['obesity'], ['irregular', 'sugar', 'level'], ['restlessness'], ['weight', 'loss'], ['blurred', 'distorted', 'vision'], ['excessive', 'hunger'], ['polyuria']], [['diarrhoea'], ['vomiting'], ['dehydration'], ['sunken', 'eyes']]

In [ ]:
import progressbar
import urllib.request

pbar = None
def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None


#urllib.request.urlretrieve(model_url, model_file, show_progress)
urllib.request.urlretrieve('https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip', show_progress)

In [7]:
!unzip "/content/glove.6B.zip" -d "/content/"

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
docs_x

[[['skin', 'rash'],
  ['dischromic', 'patches'],
  ['itching'],
  ['nodal', 'skin', 'eruptions']],
 [['chills'], ['shivering'], ['continuous', 'sneezing']],
 [['vomiting'],
  ['chest', 'pain'],
  ['acidity'],
  ['ulcers', 'tongue'],
  ['stomach', 'pain'],
  ['cough']],
 [['vomiting'],
  ['nausea'],
  ['yellowish', 'skin'],
  ['abdominal', 'pain'],
  ['loss', 'appetite'],
  ['yellowing', 'eyes'],
  ['itching']],
 [['burning', 'micturition'],
  ['skin', 'rash'],
  ['spotting', 'urination'],
  ['stomach', 'pain'],
  ['itching']],
 [['vomiting'],
  ['indigestion'],
  ['passage', 'gases'],
  ['loss', 'appetite'],
  ['abdominal', 'pain'],
  ['internal', 'itching']],
 [['extra', 'marital', 'contacts'],
  ['patches', 'throat'],
  ['high', 'fever'],
  ['muscle', 'wasting']],
 [['lethargy'],
  ['increased', 'appetite'],
  ['fatigue'],
  ['obesity'],
  ['irregular', 'sugar', 'level'],
  ['restlessness'],
  ['weight', 'loss'],
  ['blurred', 'distorted', 'vision'],
  ['excessive', 'hunger'],
  ['po

In [21]:
import numpy as np

embeddings_index = dict()
f = open('glove.6B.300d.txt', encoding="utf8")

for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [22]:
#testing embeddings

embedding_vector = embeddings_index.get("ulcer")
print(embedding_vector.shape)
arr = np.zeros(300)

(300,)


In [23]:
diseases = 0
word_count = 0
max_symptoms_count = 0
for symptoms_of_1_disease in docs_x:
  diseases += 1
  max_symptoms_count = max(max_symptoms_count, len(symptoms_of_1_disease))
  for individual_symptom in symptoms_of_1_disease:
    for word in individual_symptom:
      word_count+=1
print(diseases)
print(max_symptoms_count)
print(word_count)

256
44
3440


In [24]:
len(docs_x)

256

In [25]:
import keras
import random

class DataGen(keras.utils.all_utils.Sequence):
    def __init__(self, docs_x, output, batch_size=30):
        self.docs_x = docs_x
        self.output = output
        self.batch_size = batch_size
        self.on_epoch_end()
        self.hits=0
        self.misses=0
    
    def __getitem__(self, index):
        if((index+1)*self.batch_size > len(self.docs_x)):
            self.batch_size = len(self.docs_x) - index*self.batch_size
        
        inp_batch = self.docs_x[index*self.batch_size : (index+1)*self.batch_size]
        #print()
        inp_batch_embeddings = [] #np.zeros(shape = (self.batch_size, max_symptoms_count, 300))
        for symptoms_of_1_disease in inp_batch:
          random.shuffle(symptoms_of_1_disease)
          symptoms_embedding = np.zeros((len(symptoms_of_1_disease), 300))
          j = 0
          for individual_symptom in symptoms_of_1_disease:
            individual_symptom_embedding = np.zeros(300)
            for word in individual_symptom:
              word_embedding = embeddings_index.get(word)
              if word_embedding is not None:
                self.hits += 1
                individual_symptom_embedding += word_embedding
              else:
                self.misses += 1
            symptoms_embedding[j] = individual_symptom_embedding #new code add padding using DataGen later
            j = j + 1
          symptoms_embedding = np.pad(symptoms_embedding, [((max_symptoms_count - symptoms_embedding.shape[0]), 0), (0, 0)], mode='constant', constant_values=0)
          inp_batch_embeddings.append(symptoms_embedding)
        
        inp_batch_embeddings = np.array(inp_batch_embeddings)

        out_batch = self.output[index*self.batch_size : (index+1)*self.batch_size]
     
        return inp_batch_embeddings, out_batch
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.docs_x)/float(self.batch_size)))

In [26]:
#label encoding y axis

from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()  
y =  label_encoder.fit_transform(docs_y)
y= np.asarray(y)
print(y)

[102  15 104  51  81 180   1  76 109  38 138 177  46 179 154 174  48  73
 227 248 128 129 130 131  12 223  62 181  78 125 237 142 139 141 178  23
   0   3 231 182 144   2   3   4   5   7   6   8   9  10  11  13  14  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30 186  31  32  33
  34  35  37  36  39  40  41  42  43  44  45  47  49  50  52  53  54  55
  56  57  58  59  60  61  63  64  65  66  67  68  69  70  71  72  74  75
  77  79  80  82 183 184 185 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 176 204 205 206 207 208 209 210 211 212 213 214
 215 216 217 218 219 220 222 224 225 226 221 228 229 230 232 233 234 235
 236 238 239 240 241 242 243 244 245 246 247  83  84  85  86  87  88  89
  90  91  92  93  95  94  96  97  98  99 100 101 103 105 106 107 109 108
 110 111 112 113 114 115 116 118 119 120 121 122 123 124 126 127 128 129
 132 133 117 134 135 136 137 140 143 144 145 146 147 148 149 150 151 152
 153 155 156 157 158 159 160 161 162 163 164 165 16

In [27]:
# from keras.models import *
# from keras.layers import *

# def define_model():
#     input1 = Input(shape=(max_symptoms_count,300)) 
#     lstm1 = Bidirectional(LSTM(units=128))(input1)
#     dnn_hidden_layer1 = Dense(64, activation='relu')(lstm1) 
#     dnn_output = Dense(41, activation='softmax')(dnn_hidden_layer1)
#     model = Model(inputs=[input1],outputs=[dnn_output])
#     # compile the model
#     model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     model.summary()
#     return model
# model = define_model()

In [28]:
#Add attention layer to the deep learning network
from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN,LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context
def create_RNN_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    attention_layer = attention()(RNN_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])    
    return model    
def create_LSTM_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    LSTM_layer = LSTM(hidden_units, return_sequences=True, activation='relu')(x)
    attention_layer = attention()(LSTM_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])    
    return model    
 
#model = create_LSTM_with_attention(hidden_units=256, dense_units=128, input_shape=(max_symptoms_count,300), activation='softmax')
model = create_LSTM_with_attention(hidden_units=512, dense_units=256, input_shape=(max_symptoms_count,300), activation='softmax')
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 44, 300)]         0         
                                                                 
 lstm_1 (LSTM)               (None, 44, 512)           1665024   
                                                                 
 attention_1 (attention)     (None, 512)               556       
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
Total params: 1,796,908
Trainable params: 1,796,908
Non-trainable params: 0
_________________________________________________________________


In [29]:
batch_size = 30
train_gen = DataGen(
    docs_x,
    y,
    batch_size,
    )

train_steps = len(docs_x)//batch_size

history = model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    epochs=200, #200
    verbose=1)

Epoch 1/200
8/8 [==============================] - 5s 310ms/step - loss: 5.5477 - accuracy: 0.0163
Epoch 2/200
8/8 [==============================] - 2s 231ms/step - loss: 6.4068 - accuracy: 0.0078
Epoch 3/200
8/8 [==============================] - 2s 232ms/step - loss: 5.5037 - accuracy: 0.0547
Epoch 4/200
8/8 [==============================] - 2s 234ms/step - loss: 5.4952 - accuracy: 0.0547
Epoch 5/200
8/8 [==============================] - 2s 229ms/step - loss: 10.4519 - accuracy: 0.0391
Epoch 6/200
8/8 [==============================] - 2s 229ms/step - loss: 5.9532 - accuracy: 0.0078
Epoch 7/200
8/8 [==============================] - 2s 237ms/step - loss: 5.3928 - accuracy: 0.0781
Epoch 8/200
8/8 [==============================] - 2s 256ms/step - loss: 5.3416 - accuracy: 0.0234
Epoch 9/200
8/8 [==============================] - 2s 242ms/step - loss: 5.3842 - accuracy: 0.0078
Epoch 10/200
8/8 [==============================] - 2s 254ms/step - loss: 5.2242 - accuracy: 0.0391
Epoch 11

8/8 [==============================] - 2s 232ms/step - loss: 3.7035 - accuracy: 0.4297
Epoch 83/200
8/8 [==============================] - 2s 231ms/step - loss: 3.4337 - accuracy: 0.3594
Epoch 84/200
8/8 [==============================] - 2s 231ms/step - loss: 3.2980 - accuracy: 0.5391
Epoch 85/200
8/8 [==============================] - 2s 233ms/step - loss: 3.1271 - accuracy: 0.4609
Epoch 86/200
8/8 [==============================] - 2s 230ms/step - loss: 2.8392 - accuracy: 0.5469
Epoch 87/200
8/8 [==============================] - 2s 228ms/step - loss: 2.9263 - accuracy: 0.6172
Epoch 88/200
8/8 [==============================] - 2s 231ms/step - loss: 2.8515 - accuracy: 0.5469
Epoch 89/200
8/8 [==============================] - 2s 231ms/step - loss: 2.7338 - accuracy: 0.5391
Epoch 90/200
8/8 [==============================] - 2s 239ms/step - loss: 2.6499 - accuracy: 0.5547
Epoch 91/200
8/8 [==============================] - 2s 234ms/step - loss: 2.7104 - accuracy: 0.5547
Epoch 92/200


8/8 [==============================] - 2s 238ms/step - loss: 0.7314 - accuracy: 0.8750
Epoch 164/200
8/8 [==============================] - 2s 237ms/step - loss: 0.7479 - accuracy: 0.8828
Epoch 165/200
8/8 [==============================] - 2s 238ms/step - loss: 0.8083 - accuracy: 0.8359
Epoch 166/200
8/8 [==============================] - 2s 244ms/step - loss: 0.6789 - accuracy: 0.8984
Epoch 167/200
8/8 [==============================] - 2s 239ms/step - loss: 0.6019 - accuracy: 0.9062
Epoch 168/200
8/8 [==============================] - 2s 241ms/step - loss: 0.4692 - accuracy: 0.9219
Epoch 169/200
8/8 [==============================] - 2s 243ms/step - loss: 0.3503 - accuracy: 0.9297
Epoch 170/200
8/8 [==============================] - 2s 238ms/step - loss: 0.4000 - accuracy: 0.9453
Epoch 171/200
8/8 [==============================] - 2s 231ms/step - loss: 0.3873 - accuracy: 0.9375
Epoch 172/200
8/8 [==============================] - 2s 246ms/step - loss: 0.4343 - accuracy: 0.9141
Epoc

In [15]:
def predict_from_list(symptoms_of_1_disease):
  hits = 0
  misses = 0
  symptoms_embedding = np.zeros((max_symptoms_count-len(symptoms_of_1_disease), 300))
  for individual_symptom in symptoms_of_1_disease:
    individual_symptom_embedding = np.zeros(300)
    words = [i for i in rt.tokenize(individual_symptom) if i not in stop]
    print(words)
    for word in words:
      word_embedding = embeddings_index.get(word)
      if word_embedding is not None:
        hits += 1
        individual_symptom_embedding += word_embedding
      else:
        misses += 1
    symptoms_embedding = np.append(symptoms_embedding, [individual_symptom_embedding], axis = 0)
  #print(symptoms_embedding.shape)
  print(f"Of total words in symptoms: hits = {hits}, misses = {misses}, ratio = {hits/(hits+misses)}")
  test_data = np.array([symptoms_embedding])
  print(test_data.shape)
  pred = model.predict(test_data)
  pos = np.argmax(pred, axis=1)
  return label_encoder.inverse_transform([pos[0]])[0]

In [22]:
#lis = ["indigestion", "stiff_neck", "irritability","headache", "acidity"]

lis = [x for x in input("Enter symptoms separated by comma: ").split(",")]
print(lis)
predicted_disease = predict_from_list(lis)
print(f"\n Predicted disease: {predicted_disease}")

Enter symptoms separated by comma: loss_balance",     "feeling_sick",     "dizziness
['loss_balance"', '     "feeling_sick"', '     "dizziness']
['loss', 'balance']
['feeling', 'sick']
['dizziness']
Of total words in symptoms: hits = 5, misses = 0, ratio = 1.0
(1, 44, 300)

 Predicted disease: Vertigo


In [19]:
def predict_from_sentence(symptoms):
  print(symptoms)
  hits = 0
  misses = 0
  symptoms_embedding = np.zeros((max_symptoms_count, 300))
  words = [i for i in rt.tokenize(symptoms) if i not in stop]
  print(words)
  for word in words:
    if hits == 17:
      break
    word_embedding = embeddings_index.get(word)
    if word_embedding is not None:
      hits += 1
      symptoms_embedding[max_symptoms_count-hits] += word_embedding
    else:
      word_embedding = np.zeros(300)
      misses += 1
    
  print(symptoms_embedding.shape)
  print(f"Of total words in symptoms: hits = {hits}, misses = {misses}, ratio = {hits/(hits+misses)}")
  test_data = np.array([symptoms_embedding])
  print(test_data.shape)
  pred = model.predict(test_data)
  pos = np.argmax(pred, axis=1)
  return label_encoder.inverse_transform([pos[0]])[0]

In [21]:
sentence =  "i have loss_balance and feeling_sick and dizziness" #"i have skin rash dischromic patches itching and nodal skin eruptions"
predicted_disease = predict_from_sentence(sentence)
print(f"\n Predicted disease: {predicted_disease}")

i have loss_balance and feeling_sick and dizziness
['loss', 'balance', 'feeling', 'sick', 'dizziness']
(44, 300)
Of total words in symptoms: hits = 5, misses = 0, ratio = 1.0
(1, 44, 300)

 Predicted disease: Appendicitis


**Saving models**

In [20]:
import pickle 

filehandler = open("./chatbot_v1", 'wb') 
pickle.dump(model, filehandler)
filehandler.close()

INFO:tensorflow:Assets written to: ram://276f20d8-4a67-4c5b-b5ed-a2465d337210/assets


In [21]:
filehandler = open("./chatbot_v1", 'rb') 
mo = pickle.load(filehandler)
print(mo)
filehandler.close()

FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ram://0131364a-e395-4e1a-88bc-2363b7ee1453/variables/variables
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

In [37]:
# save_path = './chatbot_v1_model.h5'
# model.save(save_path)
# ## load tensorflow model
# model = keras.models.load_model('chatbot_v1_model.h5')